In [ ]:
import os
from huggingface_hub import InferenceClient
from dotenv import load_dotenv
load_dotenv()
import json

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


def llm(prompt):
    client = InferenceClient(
        provider="novita",
        api_key=os.getenv('HF_API_KEY'),
    )
    
    response = client.chat.completions.create(
        model='deepseek-ai/DeepSeek-V3-0324',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [ ]:
# load documents
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [15]:
query = 'how do I run kafka?'

prompt = build_prompt(query, documents)
llm(prompt)

BadRequestError: (Request ID: Root=1-6837162c-5aa797706ed0bad3706bfc54;19951ea6-506d-4523-a6cc-1f94ba1f3e5b)

Bad request: